In [129]:
# multivariate multi-step encoder-decoder lstm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from pandas import DataFrame , concat
from sklearn.metrics import mean_absolute_error , mean_squared_error
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from numpy import mean , concatenate
from math import sqrt
from pandas import read_csv
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Activation
from numpy import array , hstack
from tensorflow import keras
import tensorflow as tf

In [130]:
# read in CSV data
df = pd.read_csv('C:/Users/sweir/OneDrive/Desktop/Analysis Projects/Crude_Oil_Price_Projection/ML Resources/Cleaned_Crude_Oil_Dataset_Train2.csv')
df.head()

,Date,"Cushing, OK Crude Oil Future Contract ($/Bar.)",U.S. Field Production of Crude Oil (1k Bar.),Product Supplied of Crude Oil and Petroleum Products (1k Bar.),Gross Inputs to Refineries (1k Bar./Day),Operable Crude Oil Distillation Capacity (1k Bar./Calendar Day),Operating Crude Oil Distillation Capacity (1k Bar./Day),Idle Crude Oil Distillation Capacity (1k Bar./Day),Percent Utilization of Refinery Operable Capacity,"Cushing, OK WTI Spot Price FOB ($/Bar.)"
0,Date,"Cushing, OK Crude Oil Future Contract ($/Bar.)",U.S. Field Production of Crude Oil (1k Bar.),Product Supplied of Crude Oil and Petroleum Pr...,Gross Inputs to Refineries (1k Bar./Day),Operable Crude Oil Distillation Capacity (1k B...,Operating Crude Oil Distillation Capacity (1k ...,Idle Crude Oil Distillation Capacity (1k Bar./...,Percent Utilization of Refinery Operable Capacity,"Cushing, OK WTI Spot Price FOB ($/Bar.)"
1,12/1/1986,16.1,258926,531060,12914,15440,14662,779,83.6,16.1
2,1/1/1987,18.7,262888,517210,12725,15566,14940,625,81.8,18.7
3,2/1/1987,17.7,234880,473425,12417,15548,14748,799,79.9,17.7
4,3/1/1987,18.3,262398,501109,12237,15559,14816,743,78.6,18.3


In [131]:
# to convert the date column to type DATETIME
df['Date'] = pd.to_datetime(df['Date'])


ParserError: Unknown string format: Date

In [132]:
# view data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 10 columns):
 #   Column                                                           Non-Null Count  Dtype 
---  ------                                                           --------------  ----- 
 0   Date                                                             420 non-null    object
 1   Cushing, OK Crude Oil Future Contract ($/Bar.)                   420 non-null    object
 2   U.S. Field Production of Crude Oil (1k Bar.)                     420 non-null    object
 3   Product Supplied of Crude Oil and Petroleum Products (1k Bar.)   420 non-null    object
 4   Gross Inputs to Refineries (1k Bar./Day)                         420 non-null    object
 5   Operable Crude Oil Distillation Capacity (1k Bar./Calendar Day)  420 non-null    object
 6   Operating Crude Oil Distillation Capacity (1k Bar./Day)          420 non-null    object
 7   Idle Crude Oil Distillation Capacity (1k Bar./Day)   

In [133]:
# setting date as index
df.set_index('Date')

,"Cushing, OK Crude Oil Future Contract ($/Bar.)",U.S. Field Production of Crude Oil (1k Bar.),Product Supplied of Crude Oil and Petroleum Products (1k Bar.),Gross Inputs to Refineries (1k Bar./Day),Operable Crude Oil Distillation Capacity (1k Bar./Calendar Day),Operating Crude Oil Distillation Capacity (1k Bar./Day),Idle Crude Oil Distillation Capacity (1k Bar./Day),Percent Utilization of Refinery Operable Capacity,"Cushing, OK WTI Spot Price FOB ($/Bar.)"
Date,,,,,,,,,
Date,"Cushing, OK Crude Oil Future Contract ($/Bar.)",U.S. Field Production of Crude Oil (1k Bar.),Product Supplied of Crude Oil and Petroleum Pr...,Gross Inputs to Refineries (1k Bar./Day),Operable Crude Oil Distillation Capacity (1k B...,Operating Crude Oil Distillation Capacity (1k ...,Idle Crude Oil Distillation Capacity (1k Bar./...,Percent Utilization of Refinery Operable Capacity,"Cushing, OK WTI Spot Price FOB ($/Bar.)"
12/1/1986,16.1,258926,531060,12914,15440,14662,779,83.6,16.1
1/1/1987,18.7,262888,517210,12725,15566,14940,625,81.8,18.7
2/1/1987,17.7,234880,473425,12417,15548,14748,799,79.9,17.7
3/1/1987,18.3,262398,501109,12237,15559,14816,743,78.6,18.3
...,...,...,...,...,...,...,...,...,...
6/1/2021,71.4,338645,616115,16743,18128,17910,218,92.4,71.4
7/1/2021,72.4,351228,616714,16482,18129,17943,187,90.9,72.5
8/1/2021,67.7,347393,635828,16377,18130,17914,216,90.3,67.7


In [134]:
# singling out variables
x_1 = df['U.S. Field Production of Crude Oil (1k Bar.)']
x_2 = df['Product Supplied of Crude Oil and Petroleum Products (1k Bar.)']
x_3 = df['Gross Inputs to Refineries (1k Bar./Day)']
x_4 = df['Operable Crude Oil Distillation Capacity (1k Bar./Calendar Day)']
x_5 = df['Operating Crude Oil Distillation Capacity (1k Bar./Day)']
x_6 = df['Idle Crude Oil Distillation Capacity (1k Bar./Day)']
x_7 = df['Cushing, OK WTI Spot Price FOB ($/Bar.)']
y = df['Cushing, OK Crude Oil Future Contract ($/Bar.)']

x_1 = x_1.values
x_2 = x_2.values
x_3 = x_3.values
x_4 = x_4.values
x_5 = x_5.values
x_6 = x_6.values
x_7 = x_7.values
y = y.values

In [135]:
# rearranging variables to row x column structure 
x_1 = x_1.reshape((len(x_1), 1))
x_2 = x_2.reshape((len(x_2), 1))
x_3 = x_2.reshape((len(x_3), 1))
x_4 = x_2.reshape((len(x_4), 1))
x_5 = x_2.reshape((len(x_5), 1))
x_6 = x_2.reshape((len(x_6), 1))
x_7 = x_2.reshape((len(x_7), 1))
y = y.reshape((len(y), 1))

print("x_1.shape" , x_1.shape) 
print("x_2.shape" , x_2.shape)
print("x_3.shape" , x_3.shape) 
print("x_4.shape" , x_4.shape) 
print("x_5.shape" , x_5.shape) 
print("x_6.shape" , x_6.shape)
print("x_7.shape" , x_7.shape)
print("y.shape" , y.shape) 

x_1.shape (420, 1)
x_2.shape (420, 1)
x_3.shape (420, 1)
x_4.shape (420, 1)
x_5.shape (420, 1)
x_6.shape (420, 1)
x_7.shape (420, 1)
y.shape (420, 1)


### This model uses a MinMaxScaler

In [136]:
# creating scaler
scaler = MinMaxScaler()
x_1_scaled = scaler.fit_transform(x_1)
x_2_scaled = scaler.fit_transform(x_2)
x_3_scaled = scaler.fit_transform(x_3)
x_4_scaled = scaler.fit_transform(x_4)
x_5_scaled = scaler.fit_transform(x_5)
x_6_scaled = scaler.fit_transform(x_6)
x_7_scaled = scaler.fit_transform(x_7)
y_scaled = scaler.fit_transform(y)

ValueError: could not convert string to float: 'U.S. Field Production of Crude Oil (1k Bar.)'

In [137]:
# horizontally stack data
df_stacked = hstack((x_1_scaled, x_2_scaled, x_3_scaled, x_4_scaled, x_5_scaled, x_6_scaled, x_7_scaled, y_scaled))

print(df_stacked.shape) 

(430, 8)


In [147]:
# splitting data into samples
def splitting_sequences(sequences, num_input_steps, num_output_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + num_input_steps
		out_end_ix = end_ix + num_output_steps-1
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [148]:
# choose a number of time steps #change this accordingly
num_input_steps, num_output_steps = 12, 12

# covert into input/output
X, y = splitting_sequences(df_stacked, num_input_steps, num_output_steps)

print ("X.shape" , X.shape) 
print ("y.shape" , y.shape) 

X.shape (408, 12, 7)
y.shape (408, 12)


In [149]:
split_point = 60*6
train_X , train_y = X[:split_point, :] , y[:split_point, :]
test_X , test_y = X[split_point:, :] , y[split_point:, :]

n_features = train_X.shape[2]

In [150]:
print ("train_X.shape" , train_X.shape) 
print ("train_y.shape" , train_y.shape) 
print ("test_X.shape" , test_X.shape) 
print ("test_y.shape" , test_y.shape) 
print ("n_features" , n_features)

train_X.shape (360, 12, 7)
train_y.shape (360, 12)
test_X.shape (48, 12, 7)
test_y.shape (48, 12)
n_features 7


In [158]:
#optimizer learning rate
opt = keras.optimizers.Adam(learning_rate=0.001)

# define model
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(num_input_steps, n_features)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(num_output_steps))
model.add(Activation('linear'))
model.compile(loss='mse' , optimizer=opt , metrics=['mse'])

In [159]:
# Fit network
history = model.fit(train_X , train_y , epochs=60 , steps_per_epoch=8 , verbose=1 ,validation_data=(test_X, test_y) ,shuffle=False)

Epoch 1/60
8/8 [==============================] - 2s 52ms/step - loss: 0.1235 - mse: 0.1235 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 2/60
8/8 [==============================] - 0s 15ms/step - loss: 0.0967 - mse: 0.0967 - val_loss: 0.0643 - val_mse: 0.0643
Epoch 3/60
8/8 [==============================] - 0s 15ms/step - loss: 0.0662 - mse: 0.0662 - val_loss: 0.0331 - val_mse: 0.0331
Epoch 4/60
8/8 [==============================] - 0s 14ms/step - loss: 0.0572 - mse: 0.0572 - val_loss: 0.0283 - val_mse: 0.0283
Epoch 5/60
8/8 [==============================] - 0s 13ms/step - loss: 0.0535 - mse: 0.0535 - val_loss: 0.0178 - val_mse: 0.0178
Epoch 6/60
8/8 [==============================] - 0s 13ms/step - loss: 0.0500 - mse: 0.0500 - val_loss: 0.0161 - val_mse: 0.0161
Epoch 7/60
8/8 [==============================] - 0s 14ms/step - loss: 0.0493 - mse: 0.0493 - val_loss: 0.0167 - val_mse: 0.0167
Epoch 8/60
8/8 [==============================] - 0s 13ms/step - loss: 0.0496 - mse: 0.0496 - val

In [ ]:
# Test Data Batch 1 , Test Data Batch 2 , Test Data Batch 3
# read in CSV data
df2 = pd.read_csv('C:/Users/sweir/OneDrive/Desktop/Analysis Projects/Crude_Oil_Price_Projection/ML Resources/Cleaned_Crude_Oil_Dataset_Train.csv')
df2.head()

In [ ]:
# to convert the date column to type DATETIME
df2['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# setting date as index
df2.set_index('Date')

In [ ]:
# singling out variables and scaling
x_1_testing = df2['U.S. Field Production of Crude Oil (1k Bar.)']
x_2_testing = df2['Product Supplied of Crude Oil and Petroleum Products (1k Bar.)']
x_3_testing = df2['Gross Inputs to Refineries (1k Bar./Day)']
x_4_testing = df2['Operable Crude Oil Distillation Capacity (1k Bar./Calendar Day)']
x_5_testing = df2['Operating Crude Oil Distillation Capacity (1k Bar./Day)']
x_6_testing = df2['Idle Crude Oil Distillation Capacity (1k Bar./Day)']
x_7_testing = df2['Cushing, OK WTI Spot Price FOB ($/Bar.)']
y_testing = df2['Cushing, OK Crude Oil Future Contract ($/Bar.)']

x_1_testing = x_1_testing.values
x_2_testing = x_2_testing.values
x_3_testing = x_3_testing.values
x_4_testing = x_4_testing.values
x_5_testing = x_5_testing.values
x_6_testing = x_6_testing.values
x_7_testing = x_7_testing.values
y_testing = y_testing.values

x_1_testing_scaled = scaler.fit_transform(x_1_testing)
x_2_testing_scaled = scaler.fit_transform(x_2_testing)
x_3_testing_scaled = scaler.fit_transform(x_3_testing)
x_4_testing_scaled = scaler.fit_transform(x_4_testing)
x_5_testing_scaled = scaler.fit_transform(x_5_testing)
x_6_testing_scaled = scaler.fit_transform(x_6_testing)
x_7_testing_scaled = scaler.fit_transform(x_7_testing)
